In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import gurobipy as gp
import math
import re
from gurobipy import *

|Estudiante|Código|Correo|
|-----|-----|----|
|Juana Mejía Botero|20221512|j.mejia17|
|Daniela Ricaurte Echeverry|201822966|d.ricaurte|

# Proyecto - Entrega 5


## Lectura de datos

In [ ]:
# Datos globales del grafo
datos_globales = {}

# Nodos del grafo
nodos = []

# Demanda de potencia activa de los nodos
pd = {}
# Demanda de potencia reactiva de los nodos
qd = {}

# Resistencia de los arcos
r = {}
# Reactancia de los arcos
x = {}
# Arcos
B = []

#file_path = "C:\\Users\\danir\\Documents\\GitHub\\ProyectoFlujoEnRedes\\Sistemas_test\\datos14.txt"
file_path="/Users/juanamejia/Desktop/uni/SVII/FR/ProyectoFlujoEnRedes/Sistemas_test/datos84.txt"

In [ ]:
def getFile():
    valor=file_path.split("\\")
    last=valor[len(valor)-1]
    splitted= last.split('.')
    print(splitted[0])
    return splitted



In [ ]:
def calcZbase():
    return 1000* ((datos_globales.get('vbase')**2)/datos_globales.get('sbase'))

def lecturaDocumento():
    
    #Valores globales    
    patterns = {
        'nref': r'nref\s*=\s*([\d.]+);',
        'vref': r'vref\s*=\s*([\d.]+);',
        'vbase': r'vbase\s*=\s*([\d.]+);',
        'sbase': r'sbase\s*=\s*([\d.]+);',
        'tol': r'tol\s*=\s*([\d.^-]+);', 
        'vmin': r'vmin\s*=\s*([\d.]+);',
        'vmax': r'vmax\s*=\s*([\d.]+);',
        'zbase': r'zbase\s*=\s*([\d.]+);',
    }
    
    branch_pattern = r'ramos\s*=\s*\[(.*?)\];'
    bus_demand_pattern = r'barras\s*=\s*\[(.*?)\];'
    
    branches = []
    bus_demand = []
    
    with open(file_path, "r",encoding='latin1') as file:
        text = file.read()
    

    for key, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            if key == 'tol':
                base, exponent = match.group(1).split('^')
                datos_globales[key] = float(base) * 10 ** int(exponent)

            else:
                datos_globales[key] = float(match.group(1))
    
    datos_globales[key] = calcZbase()
    
    # Datos de ramas      
    match = re.search(branch_pattern, text, re.DOTALL)
    if match:
        branch_data = match.group(1)

        
        branch_lines = branch_data.split('\n')
        branch_lines = [line.strip() for line in branch_lines if not line.strip().startswith('%')]

       
        for line in branch_lines:
            valores = line.split()
            if len(valores) == 4:
                branch = [float(valor) for valor in valores]
                branches.append(branch)
                
    # Datos de la demanda de potencia activa y reactiva en las barras
    match = re.search(bus_demand_pattern, text, re.DOTALL)
    if match:
        bus_demand_data = match.group(1)

       
        bus_demand_lines = bus_demand_data.split('\n')
        bus_demand_lines = [line.strip() for line in bus_demand_lines if not line.strip().startswith('%')]

       
        for line in bus_demand_lines:
            valores = line.split()
            if len(valores) >= 4:
                bus = [int(valores[0]), float(valores[1]), float(valores[2]), float(valores[3])]
                bus_demand.append(bus)
    
    # Editar los valores
    valor=getFile()
    value=valor[0].split('datos')
    numNodos(int(value[1]))
    potencias(bus_demand)
    impedencia(branches)

    
def lecturaDocumento14():
    
    #Valores globales    
    patterns = {
        'nref': r'nref\s*=\s*([\d.]+);',
        'vref': r'vref\s*=\s*([\d.]+);',
        'vbase': r'vbase\s*=\s*([\d.]+);',
        'sbase': r'sbase\s*=\s*([\d.]+);',
        'tol': r'tol\s*=\s*([\d.^-]+);', 
        'vmin': r'vmin\s*=\s*([\d.]+);',
        'vmax': r'vmax\s*=\s*([\d.]+);',
        'zbase': r'zbase\s*=\s*([\d.]+);',
    }
    
    branch_pattern = r'ramos\s*=\s*\[(.*?)\];'
    bus_demand_pattern = r'barras\s*=\s*\[(.*?)\];'
    
    branches = []
    bus_demand = []
    
    
    #input('Ingresar el path del archivo \n')
    
    with open(file_path, "r",encoding='latin1') as file:
        text = file.read()
    

    for key, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            if key == 'tol':
                base, exponent = match.group(1).split('^')
                datos_globales[key] = float(base) * 10 ** int(exponent)
            else:
                datos_globales[key] = float(match.group(1))
    
    
    
    # Datos de ramas      
    match = re.search(branch_pattern, text, re.DOTALL)
    if match:
        branch_data = match.group(1)

        
        branch_lines = branch_data.split('\n')
        branch_lines = [line.strip() for line in branch_lines if not line.strip().startswith('%')]

       
        for line in branch_lines:
            valores = line.split()
            if len(valores) == 4:
                branch = [float(valor) for valor in valores]
                branches.append(branch)
                
    # Datos de la demanda de potencia activa y reactiva en las barras
    match = re.search(bus_demand_pattern, text, re.DOTALL)
    if match:
        bus_demand_data = match.group(1)

       
        bus_demand_lines = bus_demand_data.split('\n')
        bus_demand_lines = [line.strip() for line in bus_demand_lines if not line.strip().startswith('%')]

       
        for line in bus_demand_lines:
            valores = line.split()
            if len(valores) >= 4:
                bus = [int(valores[0]), float(valores[1]), float(valores[2]), float(valores[3])]
                bus_demand.append(bus)
    
    # Editar los valores
    valor=getFile()
    value=valor[0].split('datos')
    numNodos(int(value[1]))
    potencias(bus_demand)
    impedencia14(branches)
      
        
def impedencia14(branches): 
    for branch in branches:
        from_node = int(branch[0])
        to_node = int(branch[1])
        
        
        B.append( (from_node, to_node))
        B.append( ( to_node, from_node))
        r[( from_node, to_node)] = branch[2]*0.01
        r[( to_node, from_node)] = branch[2]*0.01
        x[( from_node, to_node)] = branch[3]*0.01
        x[( to_node, from_node)] = branch[3]*0.01


    
    

def numNodos(n):
    for i in range (1,n+1):
        nodos.append(i)
    


def potencias(demand):
    for bus in demand:
        pd[(bus[0])] = bus[1]
        qd[( bus[0])] = bus[2]
      
        
def impedencia(branches): 
    for branch in branches:
        from_node = int(branch[0])
        to_node = int(branch[1])
        
        
        B.append( (from_node, to_node))
        B.append( ( to_node, from_node))
        r[( from_node, to_node)] = branch[2]/datos_globales.get('zbase')
        r[( to_node, from_node)] = branch[2]/datos_globales.get('zbase')
        x[( from_node, to_node)] = branch[3]/datos_globales.get('zbase')
        x[( to_node, from_node)] = branch[3]/datos_globales.get('zbase')


In [ ]:
file_path.find("datos14") > -1

In [ ]:
if file_path.find("datos14") > -1:
    lecturaDocumento14()
else:
    lecturaDocumento()
    print(datos_globales.get("zbase"))


In [ ]:
print(nodos)
for i in nodos:
    i

## Visualización de los datos

In [ ]:
def printDatos():
    print("Nodos")
    print(nodos)
    
    print("Datos Globales")
    for key, value in datos_globales.items():
        print(f"{key}: {value}")
    
    print("Demanda de potencia Activa(pd)")
    for key, value in pd.items():
        print(key, value)


    print("Demanda de Potencia Reactiva(qd)")
    for key, value in qd.items():
        print(key, value)
    
    print("Resistencia(r) de los arcos")
    for key, value in r.items():
        print(key, value)
    
    print("Reactancia(x) de los arcos")
    for key, value in x.items():
        print(key, value)
        
    print("Arcos")
    print(B)

#printDatos()
    

## Visualizción del grafo

In [ ]:
def printGrafo():
    # Crear un grafo dirigido
    G = nx.Graph()
    
    # Añadir nodos
    for node in nodos:
        G.add_node(node)
        
    for i, j in B:
        G.add_edge(i,j)
        
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_size=1000, node_color='lightblue', width=2, font_size=15)
    plt.show()

printGrafo()


## Modelo Matemático

### Variables del Problema
- $y_{ij}$ variable binaria que representa el *estado de la llave de interconexión* entre i y j, donde esta será 1 si la llave está abierta y 0 si se encuentra cerrada.  
- $P_{ij}$ variable continua que representa el flujo de *potencia activa* entre los nodos $i$ y $j$.
- $Q_{ij}$ variable continua que representa el flujo de *potencia reactiva* entre los nodos $i$ y $j$.
- $I_{ij}$ variable continua que representa el flujo de corriente entre los nodos $i$ y $j$.
- $V_{i}$ variable continua  representa el voltaje en el nodo $i$.


### Parámetros del Problema

- $P_{i}^{D}$ representa la demanda de potencia activa en el nodo $i$ 

- $Q_{i}^{D}$  representa la demanda de potencia  reactiva en el nodo $i$.

- $R_{ij}$ representa la resistencia de la linea entre los nodos $i$ y $j$ 

- $X_{ij}$ representa la reactancia de la linea entre los nodos $i$ y $j$.

## Creación del modelo

Para le modelo $t=1$.

In [ ]:
# Create optimization model
m = gp.Model("RSD")
#Constantes

#tolerancia
tol=datos_globales.get("tol")

# b=lo que genera/consume cada nodo
b_v= 0.1


# tension minimima kV
vmin=float(datos_globales.get("vmin"))*float(datos_globales.get("vbase"))
# tension maxima kV
vmax= float(datos_globales.get("vmax"))*float(datos_globales.get("vbase"))
nref=int(datos_globales.get("nref"))

delV=vmax-vmin

# Potencia base kW
sbase= datos_globales.get("sbase")

print(vmin, vmax)
# Variables
# Estado de la llave del arco i y j
y =  m.addVars(nodos, nodos,  vtype=GRB.BINARY, name="y")

# Flujo de potencia activa entre los nodos i y j 
P =  m.addVars(nodos,nodos, vtype=GRB.CONTINUOUS,lb= 0, ub=1,  name="P")

# Flujo de potencia reactiva entre los nodos i y j 
Q = m.addVars(nodos,nodos,  vtype=GRB.CONTINUOUS,lb= 0, ub=1,  name="Q")


# Valor al cuadrado del flujo de Corriente entre los nodos i y j
Isqr = m.addVars(nodos,nodos,  vtype=GRB.CONTINUOUS, lb= 0, ub=1, name="Isqr")

# Valor al cuadrado del voltaje del nodo i
Vsqr = m.addVars(nodos, vtype=GRB.CONTINUOUS, lb=vmin**2, ub=vmax**2, name="Vsqr")

# Delta del V
deltaV= m.addVars(nodos, nodos, vtype=GRB.CONTINUOUS, lb=-delV, ub=delV, name="deltaV")


In [ ]:
m.update()

### Función Objetivo

$$
Min \sum\limits_{ij \in {B}} {{R_{ij}}I_{ij}^{2}};
$$


In [ ]:
FO = LinExpr()
for (i, j) in B:
    FO += Isqr[i,j] * r[i, j]

m.setObjective(FO, GRB.MINIMIZE)
    

### Restricciones del problema

1. Equilibrio de potencia activa: esta restricción regula que la cantidad total de potencia activa que es consumida en el sistema eléctrico sea igual a la generada y entregada al sistema, además debe ser igual a la demandada del nodo. 

$$
\sum\limits_{ki \in {B}} {{P_{ki}}}   - \sum\limits_{ij \in {B}} {\left( {{P_{ij}} + {R_{ij}}I_{ij}^{2}} \right)} + P_{i,t}^S = P_{i,t}^D;\forall i \in {N}
$$


In [ ]:
for i in nodos:    
    #restr1
    expr1 =0
    expr2 =0
    for k in nodos:
        if ((k,i) in B):
            expr1+=(P[k, i])
    
    for j in nodos:
        if ((i,j) in B):
            expr2+= P[i, j] + r[i, j] * Isqr[i, j]
    
    
    r1 = expr1 - expr2 
    
    if(i!=int(datos_globales.get("nref"))):
        m.addConstr(r1 == pd[i]/sbase, name=f"demanda_potencia_activa _{i}")
    else:
        m.addConstr(r1 <= pd[i]/sbase, name=f"demanda_potencia_activa _{i}")

2. Equilibrio de potencia reactiva: permite que la cantidad total de potencia reactiva que es consumida en el sistema eléctrico sea igual a la generada y entregada al sistema y debe igualar la demanda del nodo.

$$
\sum\limits_{ki \in {B}} {{Q_{ki,t}}}   - \sum\limits_{ij \in {B}} {\left( {{Q_{ij,t}} + {X_{ij}}I_{ij,t}^{2}} \right)}  + Q_{i,t}^S= Q_{i,t}^D;\forall i \in {N},    \forall t \in {T}
$$ 

In [ ]:
for i in nodos:
    
    expr1 =0
    expr2 =0
    for k in nodos:
        if ((k,i) in B):
            expr1+=(Q[k, i])
    
    for j in nodos:
        if ((i,j) in B):
            expr2+= Q[i, j] + x[i, j] * Isqr[i, j]
    

    r1 = expr1 - expr2 
    

    if(i!=int(datos_globales.get("nref"))):
        m.addConstr(r1 == qd[i]/sbase, name=f"demanda_potencia_reactiva _{i}")
    else:
        m.addConstr(r1 <= qd[i]/sbase, name=f"demanda_potencia_reactiva _{i}")


3. Restricción del limite de la magnitud de tensión. El voltaje debe encontrarse entre un rango especifico. 

$$
{V_{i,t}^{sqr} = V_{j,t}^{sqr} + 2\left( {{R_{ij}}{P_{ij,t}} + {X_{ij}}{Q_{ij,t}}} \right)}- Z_{ij}^2{I_{ij,t}^{sqr}} + \Delta_{ij,t}^{V}; \forall ij \in {B},  \forall  t \in {T}
$$

$$
{V_{i}^{2} = V_{j}^{2} + 2\left( {{R_{ij}}{P_{ij}} + {X_{ij}}{Q_{ij}}} \right)}- (x_{ij}^2+r_{i,j}^2){I_{ij}^{2}} + \Delta_{ij}^{V}; \forall ij \in {B}
$$

$Z_{i,j}=x_{ij}^2+r_{i,j}^2$

In [ ]:
for i,j in B: 
    expr1= 2*(P[(i, j)] * r[i,j] +Q[(i, j)] *x[i,j])
    expr2= (x[i,j]**2 + r[i,j]**2) *Isqr[i,j]
    r3= Vsqr[(j)] + expr1 - expr2 + deltaV[i,j]
    
    m.addConstr(Vsqr[i]==r3,name=f"limite de magnitud de tensión")
    


4. Restricción que permite regular la magnitud de tensión entre los nodos del sistema con respecto al estado de conexión/desconexión de las llaves de interconexión entre los nodos. 

$$
- {b^V}( {1 - {y_{ij}^B}}) \le \Delta _{ij}^V \le {b^V}( {1 -  {y_{ij}^B}});\forall ij \in B
$$
b= lo que genera/consume cada nodo


In [ ]:
for (i,j) in B: 
    expr1= -b_v *(1 - y[i, j])
    m.addConstr(expr1 <= deltaV[(i, j)] ,name=f"regular la magnitud minima de tensión entre los nodos del sistema con respecto a las llaves")
    expr2 = b_v * (1 - y[i, j])
    m.addConstr(deltaV[(i, j)]<= expr2,name=f"regular la magnitud maxima de tensión entre los nodos del sistema con respecto a las llaves")

   

5. Restricción que permite que se sigan las leyes de la conservación de energía en el sistema eléctrico y tiene en cuenta la relación entre las magnitudes de tensión, corriente y potencias activas y reactivas
$$
V_{j}^{2} I_{ij}^{2} = P_{ij}^2 +  Q_{ij}^2;\forall ij \in {B}
$$

Esta se debe de linealizar, para lograr que sea una restricción optima de LP

In [ ]:
for (i,j) in B:
    m.addConstr(Vsqr[j] * Isqr[i,j] == P[i, j] * P[i, j] + Q[i, j] * Q[i, j],
                    name=f"restriccion_cuadratica_{i}_{j}")

6. Conservación de potencia en el sistema eléctrico. 

$$
{ {\underline V }^2} \le V_{i}^{2} \le {\overline V }^2;\forall i \in N
$$

In [ ]:
for i in nodos:
    m.addConstr(vmin**2<= Vsqr[i], name =f" Conservación_minima_de_potencia_{i}")
    m.addConstr(Vsqr[i]<=vmax**2, name =f" Conservación_maxima_de_potencia_{i}")

7. Límites al cuadrado de la magnitud de la corriente entre los nodos i y j cuando la llave de interconexión se encuentre abierta. 

$$
0 \le I_{ij}^{2} \le \overline I_{ij}^2 y_{ij}^B;\forall ij \in {B}
$$

$  \overline I_{ij} = V_{max} / R_{i,j}$



In [ ]:
for (i,j) in B:
    m.addConstr(0<=Isqr[i,j], name=f"magnitud_minima_Corriente_{i}_{j}")
    m.addConstr(Isqr[i,j]<= (vmax/r[i,j])**2*y[i,j], name=f"magnitud_maxima_Corriente_{i}_{j}")
    

8. Restricción de la variable de estado de funcionamiento de los interruptores de interconexión $y_{ij,t}^B$.

$$
y_{ij,t}^B \in \left\{ {0,1} \right\};\forall ij \in {B}, \forall t \in {T}
$$

In [ ]:
for (i,j) in B:
    m.addConstr((y[i,j]+y[j,i])<=1, name=f"solo_una_una_direccion_de_flujo_entre_los_nodos_{i}_{j}")

9. Restricción de radialidad

$$\sum_{ij \in B}{ y_{ij,t}^B} = |N|- |N_{sub}|$$

Para le caso Nsub= 1, entonces

$$\sum_{ij \in B}{ y_{ij,t}^B} = |N|- 1$$


In [ ]:
print(nodos)
print(len(nodos))

In [ ]:
valor=0
print(len(nodos))

for (i,j) in B:
    valor+=y[i,j]
    
m.addConstr(valor ==(len(nodos)-1), name=f"Restriccione_de_radialidad")


In [ ]:
for i in nodos: 
    suma=0
    if (i!=int(datos_globales.get("nref"))):
        for j in nodos: 
           if (i,j) in B: 
                suma+=y[j,i]
        m.addConstr(suma >=1, name=f"conexion{i}")
    else:
        for j in nodos: 
           if (i,j) in B: 
                suma+=y[i,j]
        m.addConstr(suma >=1, name=f"conexion{i}")



In [ ]:
m.update()

In [ ]:
m.setParam("NonConvex",2)


In [ ]:
m.optimize()


In [ ]:
valor=getFile()
value=valor[0]
if m.Status == GRB.INFEASIBLE:
    m.computeIIS()
    m.write(f'{value}.ilp')
else:
    m.write(f'{value}.lp')
    m.write(f'{value}.sol')

In [ ]:
# Crear un grafo dirigido
G = nx.Graph()

def printGrafo():
   
    
    
    # Añadir nodos
    for node in nodos:
        G.add_node(node)
    
    for i, j in B:
        if y[i,j].x>0:
            G.add_edge(i,j)
        
    pos = nx.spring_layout(G)

    nx.draw(G, pos, with_labels=True, node_size=120, node_color='lightblue', width=1, font_size=10)
    nx.draw_networkx_nodes(G, pos, nodelist=[nref], node_color='red', node_size=120)
    for edge in G.edges():
        nx.draw_networkx_edges(G, pos, edgelist=[edge], width=1.0, alpha=0.5, edge_color='black', connectionstyle="arc3,rad=0.1")

    plt.show()

plt.title(f'Resultado para {value}')
printGrafo()
print(m.objval)

In [ ]:
selected_node = nref # Replace this with the node you want to check

# Check if the selected node can reach all other nodes
reachable_nodes = set(nx.node_connected_component(G, selected_node))
if reachable_nodes == set(nodos):
    print(f"The node '{selected_node}' can reach all other nodes in the graph.")
else:
    print(f"The node '{selected_node}' cannot reach all other nodes in the graph.")

In [ ]:
edges_list=[]
for i, j in B:
    if y[i,j].x>0:
        edges_list.append((i,j))

print(edges_list)

In [ ]:

GDi = nx.DiGraph()
GDi.add_edges_from(edges_list)

In [ ]:
reachable_nodes = set(nx.descendants(GDi, nref  )).union({nref  })
all_nodes = set(GDi.nodes())

if reachable_nodes == all_nodes:
    print(f"Node {nref} can reach all other nodes.")
else:
    print(f"Node {nref} cannot reach all other nodes.")